# Chapter 16 - Monte Carlo Simulation

In Chapters 14 and 15, we looked at different ways of using randomness in computations. Many of the examples we presented fall into the *__class of computation
known as Monte Carlo simulation. Monte Carlo simulation is a technique used
to approximate the probability of an event by running the same simulation multiple times and averaging the results.__*

*__Stanislaw Ulam and Nicholas Metropolis coined the term Monte Carlo simulation in 1949 in homage to the games of chance played in the casino in the Principality of Monaco__*. Ulam, who is best known for designing the hydrogen bomb
with Edward Teller, described the invention of the model as follows:

*The first thoughts and attempts I made to practice [the Monte Carlo
Method] were suggested by a question which occurred to me in 1946 as
I was convalescing from an illness and playing solitaires. The question
was what are the chances that a Canfield solitaire laid out with 52 cards
will come out successfully? After spending a lot of time trying to estimate them by pure combinatorial calculations, I wondered whether a
more practical method than “abstract thinking” might not be to lay it
out say one hundred times and simply observe and count the number of
successful plays. This was already possible to envisage with the beginning of the new era of fast computers,109 and I immediately thought of
problems of neutron diffusion and other questions of mathematical
physics, and more generally how to change processes described by certain differential equations into an equivalent form interpretable as a
succession of random operations. Later … [in 1946, I] __described the
idea to John von Neumann, and we began to plan actual calculations__.*

*__The technique was used during the Manhattan Project to predict what would
happen during a nuclear fission reaction__*, but did not really take off until the
1950s, when computers became both more common and more powerful.

Ulam was not the first mathematician to think about using the tools of probability to understand a game of chance. The history of probability is intimately
connected to the history of gambling. It is the existence of uncertainty that makes
gambling possible. And the existence of gambling provoked the development of
much of the mathematics needed to reason about uncertainty. Contributions to
the foundations of probability theory by Cardano, Pascal, Fermat, Bernoulli, de
Moivre, and Laplace were all motivated by a desire to better understand (and
perhaps profit from) games of chance.

## 16.1 Pascal's Problem

Most of the early work on probability theory revolved around games using
dice.111 Reputedly, Pascal’s interest in the field that came to be known as probability theory began when a friend asked him whether or not it would be profitable to bet that within twenty-four rolls of a pair of dice he would roll a double 6.

This was considered a hard problem in the mid-17th century. Pascal and Fermat,
two pretty smart guys, exchanged a number of letters about how to resolve the
issue, but it now seems like an easy question to answer:
    
* On the first roll the probability of rolling a 6 on each die is 1/6, so the probability of rolling a 6 with both dice is 1/36.
* Therefore, the probability of not rolling a double 6 on the first roll is 1 - 1/36 = 35/36.
* Therefore the probability of not rolling a double 6 twenty-four consecutive
times is $(35/36)^{24}$, nearly 0.51, and therefore the probability of rolling a double 6 is 
$1 - (35/36)^{24}$, about 0.49. In the long run it would not be profitable to
bet on rolling a double 6 within twenty-four rolls.

Just to be safe, let’s write a little program to simulate Pascal’s
friend’s game and confirm that we get the same answer as Pascal.

In [13]:
import random

def rollDie():
    return random.choice([1,2,3,4,5,6])

def checkPascal(numTrials):
    """Assumes numTrials>0 is an integer.
       Print an estimate of the probability of winning."""
    numWins = 0
    for i in range(numTrials):
        for j in range(24):
            d1 = rollDie()
            d2 = rollDie()
            if d1 == 6 and d2 == 6:
                numWins += 1
                break
                
    print('Probability of winning = ', numWins/numTrials)     

In [14]:
checkPascal(1000000)

Probability of winning =  0.49166


This is indeed quite close to $1 - (35/36)^{24}$.

In [15]:
1-(35/36)**(24)

0.4914038761309034

## 16.2 Pass or Don't Pass ?

## 16.3 Using Table Lookup to Improve Performance

## 16.4 Finding $\pi$

It is easy to see how Monte Carlo simulation is useful for tackling problems in
which nondeterminism plays a role. Interestingly, however, Monte Carlo simulation (and randomized algorithms in general) can be used to solve problems that
are not inherently stochastic, i.e., for which there is no uncertainty about outcomes.

Consider 𝜋. For thousands of years, people have known that there is a constant (called 𝜋 since the 18th century) such that the circumference of a circle is
equal to π*diameter and the area of the circle equal to 𝜋 * radius2. What they did
not know was the value of this constant.
One of the earliest estimates, 4*(8/9)2 =	 3.16, can found in the Egyptian
Rhind Papyrus, circa 1650 BC.

Archimedes of Syracuse (287-212 BCE) derived upper and lower bounds on
the value of 𝜋 by using a high-degree polygon to approximate a circular shape.
Using a polygon with 96 sides, he concluded that 223/71	<	𝜋 <	22/7. Giving upper and lower bounds was a rather sophisticated approach for the time. Also, if
we take his best estimate as the average of his two bounds we obtain 3.1418, an
error of about 0.0002. Not bad!

Long before computers were invented, the French mathematicians Buffon
(1707-1788) and Laplace (1749-1827) proposed using a stochastic simulation to
estimate the value of 𝜋.117 Think about inscribing a circle in a square with sides
of length 2, so that the radius, r, of the circle is of length 1.

![](montecarlo_pi.jpg)

By the definition of $\pi$, $\text{area} = \pi r^{2}$. Since r is 1, $\pi = \text{area}$. But what’s the area
of the circle? Buffon suggested that he could estimate the area of a circle by a
dropping a large number of needles (which he argued would follow a random
path as they fell) in the vicinity of the square. The ratio of the number of needles
with tips lying within the square to the number of needles with tips lying within
the circle could then be used to estimate the area of the circle.

If the locations of the needles are truly random, we know that

$ \frac{\text{needles in circle}}{\text{needles in square}} = \frac{\text{area of circle}}{\text{area of square}} $

and solving for the area of the circle:

$ \text{area of circle} = \frac{\text{area of square}*\text{needles in circle}}{\text{needles in square}} $

Recall that area of a 2 by 2 square is 4, so:

$ \text{area of circle} = \frac{4*\text{needles in circle}}{\text{needles in square}} $

In general, to estimate the area of some region R:
1. Pick an enclosing region, E, such that the area of E is easy to calculate and R	
lies completely within E.
2. Pick a set of random points that lie within E.
3. Let F be the fraction of the points that fall within R.
4. Multiply the area of E by F.

The following code contains a program that estimates $\pi$ using the Buffon-Laplace
method. For simplicity, it considers only those needles that fall in the upper
right-hand quadrant of the square.

The function throwNeedles simulates dropping a needle by first using random.random to get a pair of positive Cartesian coordinates (x and y values) representing the position of the needle with respect to the center of the square. It then
uses the Pythagorean theorem to compute the hypotenuse of the right triangle
with base x and height y. This is the distance of the tip of the needle from the
origin (the center of the square). Since the radius of the circle is 1, we know that
the needle lies within the circle if and only if the distance from the origin is no
greater than 1. We use this fact to count the number of needles in the circle.

The function getEst uses throwNeedles to find an estimate of $\pi$ by first dropping numNeedles needles, and then averaging the result over numTrials trials. It
then returns the mean and standard deviation of the trials.

The function estPi calls getEst with an ever-growing number of needles until
the standard deviation returned by getEst is no larger than precision/1.96. Under the assumption that the errors are normally distributed, this implies that 95%
of the values lie within precision of the mean.

In [20]:
def variance(X):
    """Assumes that X is a list of numbers.
       Return the variance of X."""
    mean = sum(X)/len(X)
    tot = 0.0
    for x in X:
        tot += (x-mean)**2
    return tot/len(X)

def stdDev(X):
    """Assumes that X is a list of numbers.
       Return the standard deviation of X."""
    return variance(X)**0.5

def throwNeedles(numNeedles):
    inCircle = 0
    for Needles in range(1, numNeedles+1):
        #Consider only upper-right quadrant of circle
        x = random.random()
        y = random.random()
        if (x*x + y*y)**0.5 <= 1:
            inCircle += 1
    #Since we count needles in one quadrant only, multiply result by 4.
    return 2*(inCircle/numNeedles)

def getEst(numNeedles, numTrials):
    estimates = []
    for t in range(numTrials):
        piGuess = throwNeedles(numNeedles)
        estimates.append(piGuess)
    sDev = stdDev(estimates)
    curEst = sum(estimates)/len(estimates)
    print('Est. =', str(round(curEst,5)) + ',', 'Std. dev. =', str(round(sDev,5)) + ',',
          'Needles =', numNeedles)
    return(curEst,sDev)

def estPi(precision, numTrials):
    numNeedles = 1000
    sDev = precision
    while sDev > precision/1.96:
        curEst, sDev = getEst(numNeedles, numTrials)
        numNeedles *= 2
    return curEst

In [19]:
estPi(0.01, 100)

Est. = 3.1364, Std. dev. = 0.05268, Needles = 1000
Est. = 3.1468, Std. dev. = 0.03344, Needles = 2000
Est. = 3.14147, Std. dev. = 0.02841, Needles = 4000
Est. = 3.14086, Std. dev. = 0.01533, Needles = 8000
Est. = 3.14216, Std. dev. = 0.013, Needles = 16000
Est. = 3.14073, Std. dev. = 0.0095, Needles = 32000
Est. = 3.14114, Std. dev. = 0.00709, Needles = 64000
Est. = 3.14253, Std. dev. = 0.005, Needles = 128000


3.142531250000001

As one would expect, the standard deviations decreased monotonically as we
increased the number of samples. In the beginning the estimates of the value of π
also improved steadily. Some were above the true value and some below, but each
increase in numNeedles led to an improved estimate.

Curiously, the estimate got worse when the number of needles went from
8,000 to 16,000, since 3.14135 is farther from the true value of π than is 3.14143.
However, if we look at the ranges defined by one standard deviation around each
of the means, both ranges contain the true value of 𝜋, and the range associated
with the larger sample size is smaller. Even though the estimate generated with
16,000 samples happens to be farther from the actual value of 𝜋, we should have
more confidence in its accuracy. This is an extremely important notion. It is not
sufficient to produce a good answer. We have to have a valid reason to be confident that it is in fact a good answer. And when we drop a large enough number
of needles, the small standard deviation gives us reason to be confident that we
have a correct answer. Right?

Not exactly. Having a small standard deviation is a necessary condition for
having confidence in the validity of the result. It is not a sufficient condition. The
notion of a statistically valid conclusion should never be confused with the notion of a correct conclusion.

Each statistical analysis starts with a set of assumptions. The key assumption
here is that our simulation is an accurate model of reality. Recall that the design
of our Buffon-Laplace simulation started with a little algebra demonstrating how
we could use the ratio of two areas to find the value of 𝜋. We then translated this
idea into code that depended upon a little geometry and on the randomness of
random.random.

Let’s see what happens if we get any of this wrong. Suppose, for example, we
replace the 4 in the last line of the function throwNeedles by a 2, and again run
estPi(0.01, 100). This time it prints:

In [21]:
estPi(0.01, 100)

Est. = 1.57024, Std. dev. = 0.02508, Needles = 1000
Est. = 1.57174, Std. dev. = 0.01795, Needles = 2000
Est. = 1.57056, Std. dev. = 0.01306, Needles = 4000
Est. = 1.57074, Std. dev. = 0.00923, Needles = 8000
Est. = 1.56902, Std. dev. = 0.0058, Needles = 16000
Est. = 1.57037, Std. dev. = 0.00501, Needles = 32000


1.570373749999999

The standard deviation for a mere 32,000 needles suggests that we should
have a fair amount of confidence in the estimate. But what does that really mean?
It means that we can be reasonably confident that if we were to draw more samples from the same distribution, we would get a similar value. It says nothing
about whether or not this value is close to the actual value of 𝜋. *__If you are going
to remember only one thing about statistics, remember this: a statistically valid
conclusion should not be confused with a correct conclusion!__*

Before believing the results of a simulation, *__we need to have confidence both
that our conceptual model is correct and that we have correctly implemented
that model__*. Whenever possible, one should attempt to validate results against reality. In this case, one could use some other means to compute an approximation
to the area of a circle (e.g., physical measurement) and check that the computed
value of 𝜋 is at least in the right neighborhood.

## 16.5 Some Closing Remarks About Simulation Models

For most of the history of science, theorists used mathematical techniques to
construct purely analytical models that could be used to predict the behavior of a
system from a set of parameters and initial conditions. This led to the development of important mathematical tools ranging from calculus to probability theory. These tools helped scientists develop a reasonably accurate understanding of
the macroscopic physical world.

As the 20th century progressed, the limitations of this approach became increasingly clear. Reasons for this include:

* An increased interest in the social sciences, e.g., economics, led to a desire to
construct *__good models of systems that were not mathematically tractable__*.
* As the systems to be modeled grew increasingly complex, it seemed easier to
successively refine a series of simulation models than to construct accurate analytic models.
* It is often easier to extract useful intermediate results from a simulation than
from an analytical model, e.g., to play “what if” games.
* The availability of computers made it feasible to run large-scale simulations.
Until the advent of the modern computer in the middle of the 20th century the
utility of simulation was limited by the time required to perform calculations
by hand.

*__Simulation models are descriptive, not prescriptive. They tell how a system
works under given conditions; not how to arrange the conditions to make the
system work best__*. A simulation does not optimize, it merely describes. That is
not to say that simulation cannot be used as part of an optimization process. For
example, simulation is often used as part of a search process in finding an optimal set of parameter settings.

Simulation models can be classified along three dimensions:
* Deterministic versus stochastic,
* Static versus dynamic, and
* Discrete versus continuous.

The behavior of a deterministic simulation is completely defined by the
model. Rerunning a simulation will not change the outcome. Deterministic
simulations are typically used when the system being modeled is too complex to
analyze analytically, e.g., the performance of a processor chip. Stochastic simulations incorporate randomness in the model. Multiple runs of the same model
may generate different values. This random element forces us to generate many
outcomes to see the range of possibilities. The question of whether to generate 10
or 1000 or 100,000 outcomes is a statistical question, as discussed earlier.

In a static model, time plays no essential role. The needle-dropping simulation used to estimate π in this chapter is an example of a static simulation. In a
dynamic model, time, or some analog, plays an essential role. In the series of
random walks simulated in Chapter 14, the number of steps taken was used as a
surrogate for time.

In a discrete model, the values of pertinent variables are enumerable, e.g.,
they are integers. In a continuous model, the values of pertinent variables range
over non-enumerable sets, e.g., the real numbers. Imagine analyzing the flow of
traffic along a highway. We might choose to model each individual car, in which
case we have a discrete model. Alternatively, we might choose to treat traffic as a flow, where changes in the flow can be described by differential equations. This
leads to a continuous model. In this example, the discrete model more closely resembles the physical situation (nobody drives half a car, though some cars are
half the size of others), but is more computationally complex than a continuous
one. In practice, models often have both discrete and continuous components.
For example, one might choose to model the flow of blood through the human
body using a discrete model for blood (i.e., modeling individual corpuscles) and
a continuous model for blood pressure.